In [ ]:
import pandas as pd
import os
import seaborn as sns

os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] = "yes"

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import shutil

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime as dt

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

from pathlib import Path

# ** Librerías propias
import funcion_SQL_a_DataFrame
import funcion_parametrizacion_variables

In [ ]:
resultados = funcion_parametrizacion_variables.parametrizacion_variables()
consulta_interesado_sexo = resultados[9]
consulta_interesado_npn = resultados[10]

df_consulta_interesado_sexo = funcion_SQL_a_DataFrame.sql_a_dataframe(consulta_interesado_sexo)
df_consulta_interesado_npn = funcion_SQL_a_DataFrame.sql_a_dataframe(consulta_interesado_npn)

In [ ]:
df_interesado_sexoXmunicipio = df_consulta_interesado_sexo.groupby(['codigo_dane','sexo'])['sexo'].count()
df_interesado_sexoXmunicipio = pd.DataFrame(df_interesado_sexoXmunicipio).rename(columns={'sexo':'total'}).reset_index()
df_interesado_sexoXmunicipio

In [ ]:
codigos_municipios = {'Baranoa':'08078',
    'Repelon':'08606',
    'Sabanagrande':'08634',
    'MariaLaBaja':'13442'
    }

DIRECTORIO_GRAFICO = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_5_Indicador_Social_Interlocuciones_Nivel_3\Seguimiento_Genero_IntN3\Salidas_Graficas"

# ** Aplicar tema a la salida gráfica y tipo de texto y tamaño del texto
sns.set_theme(style='whitegrid', font='Tahoma', font_scale=1)

fig, ax = plt.subplots(figsize=(8, 8))
colors = ['gold', 'yellowgreen', 'lightskyblue']

for llave, valor in codigos_municipios.items():
    for columna in df_interesado_sexoXmunicipio.columns:
        if columna == 'codigo_dane':
            globals()[f"df_{llave}_sexoXmunicipio"] = df_interesado_sexoXmunicipio[df_interesado_sexoXmunicipio[columna] == valor]

            # ! wedgeprops=dict(edgecolor='w'): Añade un borde blanco alrededor de cada porción del gráfico de pie
            wedges, texts, autotexts = plt.pie(globals()[f"df_{llave}_sexoXmunicipio"]['total'], labels=globals()[f"df_{llave}_sexoXmunicipio"]['sexo'], colors=colors, autopct='%1.1f%%', shadow=True, wedgeprops=dict(edgecolor='w'), labeldistance=1.2)
            plt.title(f'Genero por Interesado, Municipio de {llave}', pad=40, fontweight='bold', fontsize=16) # * Añadir título a la gráfica con separación, negrilla y tamaño mayor
            ax.axis('equal') # * Asegura que el pie sea un círculo

            # * Mejorar la estética de los textos
            for text in texts:
                text.set_fontsize(12)

            for autotext in autotexts:
                autotext.set_fontsize(13)
                autotext.set_fontweight('bold')
                autotext.set_color('white')
                
            NOMBRE_GRAFICO = llave + '_Sexo_X_Interesado.png'
            ruta_grafico = os.path.join(DIRECTORIO_GRAFICO, NOMBRE_GRAFICO)
            
            plt.savefig(ruta_grafico, dpi=300, bbox_inches='tight')
            plt.close()  # * Cerrar la figura para evitar superposición de gráficos

            
            DIRECTORIO_SALIDA_GRAFICA = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_5_Indicador_Social_Interlocuciones_Nivel_3\Seguimiento_Genero_IntN3\Estadisticos"
            NOMBRE_XLSX = 'Genero_X_Municipio.xlsx'

            # ** Usar ExcelWriter para guardar múltiples dataframes en diferentes hojas
            with pd.ExcelWriter(os.path.join(DIRECTORIO_SALIDA_GRAFICA, NOMBRE_XLSX), engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                globals()[f"df_{llave}_sexoXmunicipio"].to_excel(writer, sheet_name=f'GenXInt_{llave}', index=False)

In [ ]:
ruta_terrenos_w_sig = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\Analitica_UT_Consolidada\TERRENO_POR_HITO"

df_terrenos = pd.DataFrame.spatial.from_featureclass(ruta_terrenos_w_sig)

In [ ]:
df_terreno_x_sexo = pd.merge(left=df_terrenos,
    right=df_consulta_interesado_npn,
    left_on='codigo',
    right_on='numero_predial',
    how='inner')

In [ ]:
estandarizacion_columna_terrenoXsexo = ['codigo','codigo_dane','sexo','SHAPE']

RUTA_BDLOCAL = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E2_Informes_Id_FisicoJuridica\2_2_5_Indicador_Social_Interlocuciones_Nivel_3\Seguimiento_Genero_IntN3\Base_Datos\2_2_5_GeoTerrenoXGenero.gdb"
NOMBRE_FEATURE = 'Terreno_X_Genero'

df_terreno_x_sexo = df_terreno_x_sexo[estandarizacion_columna_terrenoXsexo]

df_terreno_x_sexo.spatial.to_featureclass(location=os.path.join(RUTA_BDLOCAL,NOMBRE_FEATURE))
print(f"Se crea la capa {NOMBRE_FEATURE}")